In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

#preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import mode


# pipelines
from sklearn.pipeline import Pipeline



# Customer Data

In this assignment we are going to examine some customer data gathering from my very famous internet company "the best one ever".  "The best one ever" is the best company ever that sells important things online.  In this investigation we want to find if there are any natural groups of customers in my dataset.  The first step is to just get the data in a format we can feed to our machine learning models. Once we do that, then our boss (some dude named Gilad), said he will teach us how to cluster the customers! But it turns out you need to have the _data_ formatted in some special way...? Maybe you can tell me about that!

In [2]:
# Read in the data found in "best_one_ever_database.csv" and take a look at the head and info.

In [3]:
data = pd.read_csv('best_one_ever_database.csv')

In [4]:
data.head()

,id,first_name,last_name,email,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
0,1,Jarred,Hallsworth,jhallsworth0@biblegateway.com,Non-Binary,223.109.240.148,$606.16,85349,7,NaN,3310.505486
1,2,Wayne,Robez,wrobez1@t.co,Male,73.174.113.87,$147.22,85336,9,NaN,2188.852818
2,3,Casper,Hayles,chayles2@cdc.gov,Male,223.249.117.173,$801.97,85350,4,0.597445,2640.499644
3,4,Mandie,Szepe,mszepe3@lulu.com,Female,31.143.95.228,$336.77,85364,4,NaN,2872.580992
4,5,Noam,Cecely,ncecely4@elegantthemes.com,Male,63.71.174.230,$757.45,85369,8,0.127835,2728.008966


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   int64  
 1   first_name                 2000 non-null   object 
 2   last_name                  2000 non-null   object 
 3   email                      2000 non-null   object 
 4   gender                     2000 non-null   object 
 5   ip_address                 2000 non-null   object 
 6   sales                      2000 non-null   object 
 7   zip_code                   2000 non-null   int64  
 8   total_num_of_transactions  2000 non-null   int64  
 9   prob_of_rebuy              1538 non-null   float64
 10  money_spent                1690 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 172.0+ KB


# Cleaning The Data

Taking a look at the data you have to ask yourself the questions

1. 'Which columns are useful for me to keep?'
2. 'Are all the columns usable as features?

Then you may have to do some "work" to get the column to be usable. Let's look at one column together. The first column is titled "first_name" and it seems to be the first name of each customer. Is this a usable feature? Well... not exactly in string format. So I guess I could one hot encode them into binary vectors, but even then... do I want to cluster the customers based on their first name? You can imagine some situation where clustering by name might be relevant (for example trying to guess what generation someone belonged to?) but in this case it seems like it's more of a unique identifier so it may be best to simply remove it. If every value in a column is unique (there are no duplications of the value) then we shouldn't use it as a feature because it will have a 1-1 mapping with the target variable which is not something we ever want. We want our model to learn and generalize from the features, not memorize that the name "jane" bought 5 cans of soda.

Ok, that's the first column, we vote drop! Now you have to go through each and every column and ask yourself "do I keep it? if yes, what extra work might I have to do?" 

Let's walk through it
  
  1. first_name:  this is a unique identifier so we should remove it.
  2. last_name: see above
  3. email: this is unique to an extent.  BUT if we strip the name@ portion of the email and simply keep the domain name, it could possibly aid us. Perhaps certain kinds of customers use certain email services! Worth looking into
  4. Gender: this is certainly relevant, but it's categorical data. We will need to one-hot-encode it.
  5. ip_address: We can perhaps segment the ip's into fields and use them, there maybe overlaps or correlations among different fields. Or maybe you know more about IP addresses than I do and this is totally useless
  6. sales: we certainly need this column, but we need to convert it a floating point type: remove the '$' and convert the dtype of the column
  7. zip_code: I think we can just leave this as is.
  8. prob-of_rebuy: I think we can just leave this as is.
  9. Money_spent: seems fine to me!
  

In [6]:
# drop the name columns using pandas.drop()
X = data.drop(columns=['first_name', 'last_name']) # fill in the parameters

In [7]:
# always reasonable to check `.info()` or .head() after an operation to make sure it worked
# how you thought, at least until you are comfortable with the methods / functions
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   int64  
 1   email                      2000 non-null   object 
 2   gender                     2000 non-null   object 
 3   ip_address                 2000 non-null   object 
 4   sales                      2000 non-null   object 
 5   zip_code                   2000 non-null   int64  
 6   total_num_of_transactions  2000 non-null   int64  
 7   prob_of_rebuy              1538 non-null   float64
 8   money_spent                1690 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 140.8+ KB


### Transform string columns into useful features

1. email column
2. sales column (remove $ sign)

We will use the pandas `apply` function take a function that operates on a string and apply it to the entire column. I will do the first one, and you will do the next one.

In [8]:
def strip_dollar(x):
    return x[1:]

In [9]:
# apply the function to the column and assign it back to the column (it does not work inplace)
X.sales = X.sales.apply(strip_dollar)

# cast the column to a floating point type - this is very important, otherwise it will be
# an object type column that we cannot do arithmetic on the column
X.sales = X.sales.astype('float32')

In [10]:
X.head()

,id,email,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
0,1,jhallsworth0@biblegateway.com,Non-Binary,223.109.240.148,606.159973,85349,7,NaN,3310.505486
1,2,wrobez1@t.co,Male,73.174.113.87,147.220001,85336,9,NaN,2188.852818
2,3,chayles2@cdc.gov,Male,223.249.117.173,801.969971,85350,4,0.597445,2640.499644
3,4,mszepe3@lulu.com,Female,31.143.95.228,336.769989,85364,4,NaN,2872.580992
4,5,ncecely4@elegantthemes.com,Male,63.71.174.230,757.450012,85369,8,0.127835,2728.008966


### Your Turn

Now you need to
1. write a function to strip the name portion of the email
2. Apply it to the column


In [11]:
# define functions to apply to the dataframe
def strip_emails(x):
    atindex = x.find('@')
    return x[atindex+1:]

test_email = 'thisismymail@gmail.com'
print(strip_emails(test_email))

gmail.com


In [12]:
# apply the function to the column and assign it back to the column (it does not work inplace)
X.email = X.email.apply(strip_emails)

In [13]:
X.head()

,id,email,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
0,1,biblegateway.com,Non-Binary,223.109.240.148,606.159973,85349,7,NaN,3310.505486
1,2,t.co,Male,73.174.113.87,147.220001,85336,9,NaN,2188.852818
2,3,cdc.gov,Male,223.249.117.173,801.969971,85350,4,0.597445,2640.499644
3,4,lulu.com,Female,31.143.95.228,336.769989,85364,4,NaN,2872.580992
4,5,elegantthemes.com,Male,63.71.174.230,757.450012,85369,8,0.127835,2728.008966


# Is the email column going to be worth it?
Let's take a look at this email column and decide if it could help us or not.


In [14]:
# how many unique domains are there?
counts = X.email.value_counts()
print(counts)

tiny.cc                10
xrea.com               10
furl.net               10
qq.com                 10
cbsnews.com            10
                       ..
fda.gov                 1
ed.gov                  1
devhub.com              1
creativecommons.org     1
ox.ac.uk                1
Name: email, Length: 490, dtype: int64


Based on what you think, you can either keep the column or not. It never hurts to try using it and then drop it later, it's also not a big deal if you are sure it's going to be useless. You will learn to have intuition with these things over time.

In [15]:
X.drop('email', axis=1, inplace=True)

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   int64  
 1   gender                     2000 non-null   object 
 2   ip_address                 2000 non-null   object 
 3   sales                      2000 non-null   float32
 4   zip_code                   2000 non-null   int64  
 5   total_num_of_transactions  2000 non-null   int64  
 6   prob_of_rebuy              1538 non-null   float64
 7   money_spent                1690 non-null   float64
dtypes: float32(1), float64(2), int64(3), object(2)
memory usage: 117.3+ KB


## Splitting the IP Address
We now need to split up the IP address, we will use Pandas's built in str method for this.
Again, I'm not even sure this is a good idea, but it's certainly fine practice to learn how to spit up strings in pandas!

In [17]:
# We are going to use pandas str method here, pay close attention!

X[['first_ip','second_ip','third_ip','fourth_ip']] = X.ip_address.str.split(pat=".", expand=True)

In [18]:
# now we cast the columns as floats, because we always need numbers for our models!
X[['first_ip','second_ip','third_ip','fourth_ip']] = X[['first_ip','second_ip','third_ip','fourth_ip']].astype('float32')
# we also drop the original column
X.drop('ip_address', axis=1, inplace=True)

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   int64  
 1   gender                     2000 non-null   object 
 2   sales                      2000 non-null   float32
 3   zip_code                   2000 non-null   int64  
 4   total_num_of_transactions  2000 non-null   int64  
 5   prob_of_rebuy              1538 non-null   float64
 6   money_spent                1690 non-null   float64
 7   first_ip                   2000 non-null   float32
 8   second_ip                  2000 non-null   float32
 9   third_ip                   2000 non-null   float32
 10  fourth_ip                  2000 non-null   float32
dtypes: float32(5), float64(2), int64(3), object(1)
memory usage: 132.9+ KB


# One Hot Encoding

Ok we are almost done, we just have to convert the gender column into something integer that we can use. We will use one-hot-encoding since gender is a categorical variable.

Pandas has a `get_dummies()` function that will be very useful.
I'm going to let you look it up and learn how to use it.


In [20]:
# check the gender distribution, just because
X.gender.value_counts()

Male          775
Female        725
Non-Binary    500
Name: gender, dtype: int64

In [21]:
gender_list = ['Male', 'Female', 'Non-Binary']

In [22]:
encoding = pd.get_dummies(X.gender)
X = pd.concat((X,encoding), axis=1)
X.drop(['gender'], axis=1, inplace=True)
X.head()

,id,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent,first_ip,second_ip,third_ip,fourth_ip,Female,Male,Non-Binary
0,1,606.159973,85349,7,NaN,3310.505486,223.0,109.0,240.0,148.0,0,0,1
1,2,147.220001,85336,9,NaN,2188.852818,73.0,174.0,113.0,87.0,0,1,0
2,3,801.969971,85350,4,0.597445,2640.499644,223.0,249.0,117.0,173.0,0,1,0
3,4,336.769989,85364,4,NaN,2872.580992,31.0,143.0,95.0,228.0,1,0,0
4,5,757.450012,85369,8,0.127835,2728.008966,63.0,71.0,174.0,230.0,0,1,0


In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   int64  
 1   sales                      2000 non-null   float32
 2   zip_code                   2000 non-null   int64  
 3   total_num_of_transactions  2000 non-null   int64  
 4   prob_of_rebuy              1538 non-null   float64
 5   money_spent                1690 non-null   float64
 6   first_ip                   2000 non-null   float32
 7   second_ip                  2000 non-null   float32
 8   third_ip                   2000 non-null   float32
 9   fourth_ip                  2000 non-null   float32
 10  Female                     2000 non-null   uint8  
 11  Male                       2000 non-null   uint8  
 12  Non-Binary                 2000 non-null   uint8  
dtypes: float32(5), float64(2), int64(3), uint8(3)
me

# Data Preprocessing Stage 2

Ok, we are done with stage 1 - we have converted everything into numeric features and dropped all the unneccessary things. Please double check that! Make sure all features are numeric (a check with .info() should do the trick).

However we do have missing values. Which two columns have missing values?
How many values are missing?
What should we do about those missing values?

You can either impute (fill in) the missing values, or drop the rows which contain them. The choice is up to you!
Either way, you should practice both methods. This way you can practice coding both solutions.

Note:
The `DataFrame.fillna()` method essentially assumes that you are using timeseries data. We are not, so I wouldn't use this. In order to impute simple values, you can use numpy easily, but... I'm lazy and would probably use the scikit-learn implementation.

https://scikit-learn.org/stable/modules/impute.html

In [24]:
from sklearn.impute import SimpleImputer

In [25]:
simple_imputer = SimpleImputer() #By default it looks for nan and replaces them with mean so no need to put in args

In [26]:
filled_in_X = simple_imputer.fit_transform(X)

In [29]:
new_X = pd.DataFrame(filled_in_X, columns=X.columns)

In [30]:
new_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         2000 non-null   float64
 1   sales                      2000 non-null   float64
 2   zip_code                   2000 non-null   float64
 3   total_num_of_transactions  2000 non-null   float64
 4   prob_of_rebuy              2000 non-null   float64
 5   money_spent                2000 non-null   float64
 6   first_ip                   2000 non-null   float64
 7   second_ip                  2000 non-null   float64
 8   third_ip                   2000 non-null   float64
 9   fourth_ip                  2000 non-null   float64
 10  Female                     2000 non-null   float64
 11  Male                       2000 non-null   float64
 12  Non-Binary                 2000 non-null   float64
dtypes: float64(13)
memory usage: 203.2 KB


## Comment:

Above fills in the missing data for 'money_spent' and 'prob_of_rebuy' using the mean average. One might expect 'total_num_of_transactions' to have an effect on these variables (e.g. if a customer has bought more stuff, it's more likely that they've spent more). So it might be better to fill in the missing values using a model trained from that data, instead of always putting in the mean.

The mean might also be a poor indicator if our data is very spread out. For example, it might be the case that we have a few customers who spent loads, while the rest spend little with few users in between. In this scenario, just using the mean won't help us decide which category that customer belongs to. As always, it would be important to look at the data and see if this is the case (before deciding what strategy to employ for filling in the missing values).

Another thought: when we fill in missing values, our new data doesn't 'remember' which data is real and which data is guessed. It might be a good idea to look into the possibility of weighting our data's reliability and somehow getting this into the model. In this way, the model would make decisions/structure itself in a way which is biased towards our 'real' data (instead of putting too much focus on the guessed data). For example, in a decision tree classifier with the gini coefficient, we could categorise the data as 'real'/'guessed' and incorporate this into the gini formula to prioritize splitting the real data over the guessed data.